# Financial Data Collection and Storage #

## Env Setup ##

In [1]:
# lib imports
import re
import requests
import pandas as pd
import yfinance as YF
from fredapi import Fred
import mysql.connector as SQL
from utils import get_tickers


## Stocks and Economic Indicators ##

In [2]:
# define key variables
fred_key = "7c1f3f043ef85e198d4c6c17e0d8109e"
tickers = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "JPM",   # JPMorgan Chase
    "WMT",   # Walmart
    "META",  # Meta Platforms
    "AMZN",  # Amazon
    "LMT",   # Lockheed Martin
    "GOOGL", # Alphabet
    "GS",    # Goldman Sachs
    "XOM",   # ExxonMobil
    "CVX",   # Chevron
    "PG",    # Procter & Gamble
    "KO",    # Coca-Cola
    "FDX",   # FedEx
    "UNP",   # Union Pacific
    "JNJ",   # Johnson & Johnson
    "PFE"    # Pfizer
]


In [3]:
# pull stocks data
stocks = get_tickers(tickers, period='max')
stocks.reset_index(inplace=True)

In [4]:
# pull fred data
fred_api = Fred(fred_key)
cpi = fred_api.get_series('CPIAUCNS')
unrate = fred_api.get_series("Unrate")
csi = fred_api.get_series("UMCSENT")
dff = fred_api.get_series("DFF")
m30 = fred_api.get_series('MORTGAGE30US')

In [5]:
# join fred data
fred = pd.DataFrame(pd.concat([cpi, unrate, csi, dff, m30], axis=1)).reset_index()
fred.rename(columns={'index': 'Date', 0: 'cpi', 1: 'unrate', 2:'csi', 3: 'dff', 4: '30Y_mort'}, inplace=True)

In [6]:
# Convert stock dates to naive datetime (removing timezone)
stocks['Date'] = stocks['Date'].dt.tz_localize(None)

# Convert fred dates to ensure consistency (though likely already naive)
fred['Date'] = fred['Date'].dt.tz_localize(None)

# merge data on date
fi_df = pd.merge(stocks, fred, on='Date', how='left')

# backfill monthly indicators to prevent NaN values
fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']] = fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']].ffill()
fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']] = fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']].bfill()

# ensure consistency
fi_df.columns = fi_df.columns.str.lower()
# place ticker at pos 0
fi_df.insert(0, 'ticker', fi_df.pop('ticker'))

## SP500 and Economic Indicators ##

In [7]:
# define vars
fred_key = "7c1f3f043ef85e198d4c6c17e0d8109e"

In [8]:
# pull fred and sp500 data
fred_api = Fred(fred_key)
cpi = fred_api.get_series('CPIAUCNS')
unrate = fred_api.get_series("Unrate")
csi = fred_api.get_series("UMCSENT")
dff = fred_api.get_series("DFF")
m30 = fred_api.get_series("MORTGAGE30US")
sp5c = YF.download("^GSPC", "1900-01-01")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [9]:
# reduce to single index
sp5c.columns = ['_'.join(col) for col in sp5c.columns]
sp5c.columns = [re.sub(r"\^", "", col) for col in sp5c.columns]
sp5c.reset_index(inplace=True)

In [10]:
# join fred data
fred = pd.DataFrame(pd.concat([cpi, unrate, csi, dff, m30], axis=1)).reset_index()
fred.rename(columns={'index': 'Date', 0: 'cpi', 1: 'unrate', 2:'csi', 3: 'dff', 4: '30Y_mort'}, inplace=True)

In [11]:
# merge data on date
sp500 = pd.merge(sp5c, fred, on='Date', how='left')

# backfill monthly indicators to prevent NaN values
sp500[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']] = sp500[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']].ffill()
sp500[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']] = sp500[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']].bfill()

# ensure consistency
sp500.columns = sp500.columns.str.lower()
sp500.head()

,date,close_gspc,high_gspc,low_gspc,open_gspc,volume_gspc,cpi,unrate,csi,dff,30y_mort
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,0,17.1,4.0,82.0,1.13,7.33
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,0,17.1,4.0,82.0,1.13,7.33
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,0,17.1,4.0,82.0,1.13,7.33
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,0,17.1,4.0,82.0,1.13,7.33
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,0,17.1,4.0,82.0,1.13,7.33


## Stock Storage ##

In [12]:
# set up connection
conn = SQL.connect(host='localhost',
                   user='root',
                   password='root',
                   database='fi_db')

cursor = conn.cursor()

In [13]:
cursor.execute("DROP DATABASE IF EXISTS fi_db;")

In [14]:
# create db and activate db
cursor.execute("CREATE DATABASE IF NOT EXISTS fi_db;")
cursor.execute("USE fi_db;")

# create table with schema
cursor.execute("""
               CREATE TABLE IF NOT EXISTS hist(
                id INT AUTO_INCREMENT PRIMARY KEY,
                ticker VARCHAR(10),
                date DATETIME,
                open FLOAT,
                high FLOAT,
                low FLOAT,
                close FLOAT,
                volume FLOAT,
                dividends FLOAT,
                `stock splits` FLOAT,
                cpi FLOAT,
                unrate FLOAT,
                csi FLOAT,
                dff FLOAT,
                30y_mort FLOAT);
                """)

In [15]:
# push fi_df to sql
for idx, row in fi_df.iterrows():
    cursor.execute("""
                   INSERT INTO fi_db.hist (ticker, date, open, high, low,
                   close, volume, dividends, `stock splits`, cpi, unrate, csi, dff, 30y_mort)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                   """,
                   (row['ticker'], row['date'], row['open'], row['high'], row['low'],
                    row['close'], row['volume'], row['dividends'], row['stock splits'],
                    row['cpi'], row['unrate'], row['csi'], row['dff'], row['30y_mort'])
            )

In [16]:
# finalize the transfer
try:
    conn.commit()
    cursor.close()
    conn.close()
    print("Data successfully transferred!")
except Exception as e:
    print(e)

Data successfully transferred!


## SP500 Storage ##

In [17]:
# set up connection
conn = SQL.connect(host='localhost',
                   user='root',
                   password='root',
                   database='fi_db')

cursor = conn.cursor()

In [18]:
# create db and activate db
cursor.execute("USE fi_db;")

# create table with schema
cursor.execute("""
               CREATE TABLE IF NOT EXISTS sp500(
                id INT AUTO_INCREMENT PRIMARY KEY,
                date DATETIME,
                close_gspc FLOAT,
                high_gspc FLOAT,
                low_gspc FLOAT,
                open_gspc FLOAT,
                volume_gspc FLOAT,
                cpi FLOAT,
                unrate FLOAT,
                csi FLOAT,
                dff FLOAT,
                30y_mort FLOAT);
                """)

In [19]:
# push fi_df to sql
for idx, row in sp500.iterrows():
    cursor.execute("""
                   INSERT INTO fi_db.sp500 (date, close_gspc, high_gspc, low_gspc,
                   open_gspc, volume_gspc, cpi, unrate, csi, dff, 30y_mort)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                   """,
                   (row['date'], row['close_gspc'], row['high_gspc'], row['low_gspc'],
                    row['open_gspc'], row['volume_gspc'],row['cpi'], row['unrate'],
                    row['csi'], row['dff'], row['30y_mort'])
            )

In [20]:
# finalize the transfer
try:
    conn.commit()
    cursor.close()
    conn.close()
    print("Data successfully transferred!")
except Exception as e:
    print(e)

Data successfully transferred!
